In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from typing import TypedDict, Literal
from pydantic import Field, BaseModel
from langgraph.graph import StateGraph, START, END
from IPython.display import display, Image

# Load environment variables
os.environ["LANGSMITH_PROJECT"] = "AgenticAIworkspace"
load_dotenv()

groq_key = os.getenv("GROQ_API_KEY")

# Initialize the LLM
llm = ChatGroq(api_key=groq_key, model="llama-3.1-8b-instant")